In [141]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM

# Load the dataset from CSV
df = pd.read_csv('C:/Users/DELL/Desktop/ENDSEM_EE798/STOCK_INDEX.csv')

# Extract the 'Close' column for prediction
close_prices = df.Close

#Dealing with missing values
close_prices=close_prices.ffill()
close_prices=close_prices.values.reshape(-1, 1)

# Normalize the data between 0 and 1
scaler = MinMaxScaler()
scaled_close_prices = scaler.fit_transform(close_prices)

# Split the dataset into training and testing sets
train_size = int(len(scaled_close_prices) * 0.75)
train_data, test_data = scaled_close_prices[:train_size], scaled_close_prices[train_size:]

# Prepare the data for training and testing
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps - 1):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

time_steps = 5  # Define the number of time steps
X_train, y_train = prepare_data(train_data, time_steps)
X_test, y_test = prepare_data(test_data, time_steps)

# Build and train the GRU model
tf.keras.backend.clear_session()
model=Sequential()
model.add(GRU(32,return_sequences=True,input_shape=(time_steps,1)))
#model.add(GRU(32,return_sequences=True))
model.add(GRU(32))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=40,batch_size=32)

Epoch 1/40
63/63 [==============================] - 12s 29ms/step - loss: 0.0139 - val_loss: 6.9134e-04
Epoch 2/40
63/63 [==============================] - 1s 10ms/step - loss: 7.2177e-04 - val_loss: 4.3629e-04
Epoch 3/40
63/63 [==============================] - 1s 10ms/step - loss: 6.2114e-04 - val_loss: 6.1470e-04
Epoch 4/40
63/63 [==============================] - 1s 10ms/step - loss: 5.4061e-04 - val_loss: 4.5512e-04
Epoch 5/40
63/63 [==============================] - 1s 10ms/step - loss: 4.7844e-04 - val_loss: 4.8272e-04
Epoch 6/40
63/63 [==============================] - 1s 10ms/step - loss: 4.6220e-04 - val_loss: 4.2162e-04
Epoch 7/40
63/63 [==============================] - 1s 11ms/step - loss: 4.7005e-04 - val_loss: 6.2185e-04
Epoch 8/40
63/63 [==============================] - 1s 12ms/step - loss: 4.1631e-04 - val_loss: 4.0656e-04
Epoch 9/40
63/63 [==============================] - 1s 11ms/step - loss: 4.1221e-04 - val_loss: 4.0012e-04
Epoch 10/40
63/63 [=====================

In [143]:
# Evaluate the model on the testing set
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)

21/21 [==============================] - 0s 4ms/step - loss: 9.4746e-04
Test Loss: 0.0009474620455875993


In [66]:
# Print the model weights
print('GRU Model Weights:')
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        print(layer.name)
        for i, w in enumerate(weights):
            print(f'Weight {i+1}:')
            print(w)
            print()

GRU Model Weights:
gru
Weight 1:
[[ 0.03673842 -0.2541932  -0.18870154 -0.37977126  0.09899077  0.22344506
  -0.29300737 -0.10636775 -0.22258985 -0.3238251  -0.3909641  -0.4366188
  -0.07488832 -0.06782348 -0.34161285  0.23441777 -0.2438606  -0.1625027
  -0.10503617  0.08998819 -0.05272721 -0.38902855  0.0825983  -0.3561489
  -0.19185333 -0.12439746 -0.06893912 -0.32319483 -0.03962326 -0.17252082
  -0.03949542  0.04897941  0.03661498 -0.15162876 -0.02085917 -0.22744249
  -0.0944577   0.08074562 -0.119087   -0.2879082  -0.03055935 -0.14756827
   0.00571496 -0.26201427  0.12540919 -0.11574994  0.06263305  0.1759757
  -0.07836733 -0.19065611 -0.04881701  0.20613928 -0.00412761 -0.09374435
  -0.12909678  0.07258523 -0.06860534 -0.24055369  0.08054116 -0.11945159
   0.0134093  -0.14830898  0.05055749  0.1020399  -0.2546114  -0.16031237
  -0.15695152 -0.19942491 -0.22428207 -0.00723124  0.10603762  0.27169102
   0.25832462 -0.23792548  0.26731256  0.22173168  0.19397214  0.07261214
   0.1420

In [133]:
# Save the model weights            
model.save_weights('MODEL1.h5')
print('GRU model weights saved successfully.')

GRU model weights saved successfully.
